In [1]:
USER_KEY = "cc3948f0cd364a9f684a1fe78b9229ba"      #  Input your User key
DATA_KEY = "a71eaf04-802f-40be-93c2-5bee2548f4db"  #  Fixed Data Api key

print("USER_KEY:", USER_KEY if USER_KEY else "Not found")
print("DATA_KEY:", DATA_KEY if DATA_KEY else "Not found")

USER_KEY: cc3948f0cd364a9f684a1fe78b9229ba
DATA_KEY: a71eaf04-802f-40be-93c2-5bee2548f4db


In [2]:
import requests

root_url = f'https://aifapbt.fin.cloud.ainode.ai/'

headers = {"API-KEY": USER_KEY}
requests.get(root_url, headers=headers).json()

{'message': 'API Server is Running.',
 'status': 200,
 'User status': 'active',
 'Expires At': '2025-10-01T00:00:00',
 'Level': 1}

In [3]:
import os
strategy_name = "reversion"

# Upload or Update strategy file
endpoint = 'upload/strategy'
url = root_url + endpoint

base_path = "/Users/seongmin/Desktop/neomatrix_trading/futures"
file_path = os.path.join(base_path, strategy_name, strategy_name + ".py")

params = {'tradeType':'futures', 'overwrite': True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

📂 Upload Response: {'message': "'reversion' Strategy Updated Successfully!"}


In [4]:
# Check strategy file upload
endpoint = 'upload/check/strategy'
url = root_url + endpoint

params = {"tradeType": "futures", "strategy_name": strategy_name}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# strategy.py

import requests
import pandas as pd
import time
from datetime import datetime
import numpy as np

def strategy(df, config_dict):
    
    data_url = "https://crypto.fin.cloud.ainode.ai/a71eaf04-802f-40be-93c2-5bee2548f4db/get/info/coin"
    response = requests.get(data_url)
    data = response.json()

    symbols = []
    for item in data['data']:
        symbols.append(item['coin_nm'] + 'USDT')

    def get_bitget_history_ohlcv(symbols, days_to_fetch, granularity='1D', product_type='umcbl'):
        """
        Bitget API를 사용하여 여러 종목의 일봉 OHLCV 데이터를 가져옵니다.
        API의 200개 제한을 해결하기 위한 페이지네이션 로직을 포함합니다.

        Args:
            symbols (list): 데이터를 가져올 심볼 리스트 (예: ['BTCUSDT', 'ETHUSDT'])
            days_to_fetch (int): 가져올 데이터의 날짜 수 (예: 500)
            granularity (str): 캔들 주기 ('1D'는 일봉)
            product_type (str): 상품 타입 ('umcbl'은 USDT 선물)

        Returns:
            pandas.DataFrame: MultiIndex 컬럼을 가진 OHLCV 데이터프레임.
                            (레벨 0: 심볼, 레벨 1: o

In [5]:
# Upload or Update strategy-config file
endpoint = 'upload/strategy-config'
url = root_url + endpoint

strategy_config_name = strategy_name + "_config"
base_path = "/Users/seongmin/Desktop/neomatrix_trading/futures/"
file_path = os.path.join(base_path, strategy_name, strategy_config_name + ".py")

params = {"tradeType": "futures", "overwrite": True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

📂 Upload Response: {'message': "'reversion' Strategy-Config Updated Successfully!"}


In [6]:
# Check strategy-config file upload
endpoint = 'upload/check/strategy-config'
url = root_url + endpoint

params = {"tradeType": "futures", "strategy_name": strategy_name}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# strategy_config.py

strategy_config = {
    # 1. Price-based mean-reversion core
    "median_lookback": 60,       # 롤링 중간값 계산 기간
    "std_lookback": 60,          # 롤링 표준편차 계산 기간

    # 2. Volume Gate (거래량 필터)
    "volume_z_lookback": 20,       # 총 거래량 z-score 계산 기간
    "volume_z_threshold": 1.5,     # z-score 필터링 임계값

    # 3. Cross-sectional ranking and selection (교차 순위 선정)
    "long_percentile": 0.10,     # 롱 포지션 진입을 위한 하위 순위 백분율
    "short_percentile": 0.10,    # 숏 포지션 진입을 위한 상위 순위 백분율

    # 4. Volatility-targeted sizing (변동성 기반 비중 조절)
    "vol_lookback": 30,          # 수익률 변동성 계산 기간
    "vol_target_scaler": 0.0005, # 목표 변동성에 맞추기 위한 스케일러

    # 5. Risk caps and turnover control (리스크 관리)
    "weight_clip": 0.005,        # 개별 자산의 최대 비중 (0.5%)

    # 5b. Draw-down guard (손실 제한)
    "dd_lookback": 5,            # 손실률 계산 기간
    "dd_threshold": 0.02         # 손실 제한 발동 임계값 (2%)
}


In [7]:
# Upload or Update system-config file
endpoint = "upload/system-config"
url = root_url + endpoint

system_config_name = "futures_config"
base_path = "/Users/seongmin/Desktop/neomatrix_trading/futures"
file_path = os.path.join(base_path, system_config_name + ".py")

params = {"tradeType": "futures", "overwrite": True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

📂 Upload Response: {'message': "'Futures' System-Config Updated Successfully!"}


In [8]:
# Check system-config file upload
endpoint = "upload/check/system-config"
url = root_url + endpoint

params = {"tradeType": "futures"}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# ==========================
# Required System Settings
# ==========================
system_config = {
    "data_apikey": "a71eaf04-802f-40be-93c2-5bee2548f4db",
    "tz_str": "Asia/Seoul", 
    "timeframe": "1min",
    "orderType": "market",
    "base_symbol": "BTCUSDT",
    "productType": "usdt-futures",
    "posMode": "hedge_mode",
    "holdSide": "long",                   
    "marginMode": "crossed",
    "marginCoin": "usdt",
    "symbols": ['BTCUSDT', 'ETHUSDT', 'LTCUSDT', 'TRXUSDT', 'DOTUSDT', 'DOGEUSDT', 'XRPUSDT', 'BCHUSDT', 'ADAUSDT'],
    "leverage": 5,
    "trading_hours": 6,
    "total_allocation": 0.4, 
}


# ==========================
# Rebalancing Trade Parameters
# ==========================
rebalancing_config = {
    "rebalancing_interval_hours": 1, # Rebalancing cycle (hours)
}



In [12]:
# run-system
endpoint = 'command/run-system'
url = root_url + endpoint

data = {
    "tradeType": "futures",
    "strategy_name": strategy_name,
    "trade_mode": "rebalancing",
    "trade_env": "live"
}

response = requests.post(url, headers=headers, json=data)

try:
    session_id = response.json()['session_id']
    print('session_id :',response.json()['session_id'])
    print('dashboard :',response.json()['dashboard_url'])
except:
    print('Error :',response.json()['message']['message'])

    session_id = response.json()['message']['session_id']
    print('session_id id :',session_id)

    print('dashboard :',response.json()['message']['dashboard_url'])

session_id : a80ee024
dashboard : https://aifapbt.fin.cloud.ainode.ai/dashboard/live_futures_trading_dashbaord.html


In [11]:
# Terminate process
# You must execute terminate to force liquidation of the current process.
endpoint = 'command/terminate'
url = root_url + endpoint

params = {"session_id": f"{session_id}", "tradeType": 'futures'}

response = requests.get(url, headers=headers, params=params)
print(response.json())

{'message': 'That session does not exist.'}
